In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain_core.runnables import RunnablePassthrough
from pendulum import DateTime, Date
from datetime import date

In [2]:
from pydantic import BaseModel, Field
from decimal import Decimal
from enum import StrEnum


class ExpenseType(StrEnum):
    INCOME = "income"
    EXPENSE = "expense"


class ExpenseResult(BaseModel):
    amount: Decimal = Field(
        default=Decimal("0.00"),
        description="The amount of the expense from the user message.",
    )
    dt: date = Field(
        default_factory=lambda: DateTime.now().date(),
        description="The date of the expense from the user message. defaults from the current date.",
    )

    type: ExpenseType = Field(
        description="The type of the expense, either income or expense.",
    )

    def __str__(self):
        return f"{self.type.value.capitalize()} of {self.amount} on {self.dt}"

    def __repr__(self):
        return self.__str__()


class ResultSchema(BaseModel):
    expenses: list[ExpenseResult] = Field(description="The result of the expenses extraction from the user message.")

In [3]:
parameters = dict(
    base_url="http://192.168.81.2:11434/v1",
    api_key="ollama",  # required, but unused,
    model="qwen3:4B",
    temperature=0.0,
    max_tokens=10240,
)
client = ChatOpenAI(**parameters).with_structured_output(
    schema=ResultSchema,
    include_raw=True,
    strict=True,
)

In [4]:
client

{
  raw: RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x105f87a40>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x111651e20>, root_client=<openai.OpenAI object at 0x105f87c20>, root_async_client=<openai.AsyncOpenAI object at 0x107358d40>, model_name='qwen3:4B', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='http://192.168.81.2:11434/v1', max_tokens=10240), kwargs={'response_format': <class '__main__.ResultSchema'>, 'ls_structured_output_format': {'kwargs': {'method': 'json_schema', 'strict': True}, 'schema': {'type': 'function', 'function': {'name': 'ResultSchema', 'description': '', 'parameters': {'properties': {'expenses': {'description': 'The result of the expenses extraction from the user message.', 'items': {'properties': {'amount': {'anyOf': [{'type': 'number'}, {'type': 'string'}], 'default': '0.00', 'description': 'The amount of 

In [5]:
__SYSTEM_PROMPT = """
You are a helpful assistant that extracts expenses from user messages.
You will receive a user message and you need to extract the expenses from it.
The expenses can be of two types: income and expense.
You will return the expenses in a structured format.

ignore the not related sentences to the task, and extract the expenses and income from the user message.
The accuracy of the task is important. You must extract the expenses and income from the user message accurately.

You are an assistant specialized in extracting financial entries from free-form user messages.
Your job is to identify all expenses and incomes mentioned, determine their dates and amounts, and output them in a precise JSON format.

"""


prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(__SYSTEM_PROMPT),
        SystemMessagePromptTemplate.from_template("/no_think"),
        HumanMessagePromptTemplate.from_template("{message}"),
        HumanMessagePromptTemplate.from_template("current DATETIME with time zone is {current_datetime}"),
    ]
)


In [6]:
prompt_template.format_prompt(
    current_datetime=DateTime.now("Asia/Seoul").to_iso8601_string(),
    message="""
    담배값으로 2만원을 썼고
    커피값으로 5천원을 씀
    점심 값으로 1만원을 썼어
    그리고 어제 저녁으로 3만원을 썼어
    """,
).to_messages()

[SystemMessage(content='\nYou are a helpful assistant that extracts expenses from user messages.\nYou will receive a user message and you need to extract the expenses from it.\nThe expenses can be of two types: income and expense.\nYou will return the expenses in a structured format.\n\nignore the not related sentences to the task, and extract the expenses and income from the user message.\nThe accuracy of the task is important. You must extract the expenses and income from the user message accurately.\n\nYou are an assistant specialized in extracting financial entries from free-form user messages.\nYour job is to identify all expenses and incomes mentioned, determine their dates and amounts, and output them in a precise JSON format.\n\n', additional_kwargs={}, response_metadata={}),
 SystemMessage(content='/no_think', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='\n    담배값으로 2만원을 썼고\n    커피값으로 5천원을 씀\n    점심 값으로 1만원을 썼어\n    그리고 어제 저녁으로 3만원을 썼어\n    ', additional

In [7]:
results = client.invoke(
    prompt_template.format_prompt(
        current_datetime=DateTime.now("Asia/Seoul").to_iso8601_string(),
        message="""
        담배값으로 2만원을 썼고
        커피값으로 5천원을 씀
        점심 값으로 1만원을 썼어
        그리고 어제 저녁으로 3만원을 썼어
        """,
    ).to_messages(),
)

In [12]:
# results['raw']
# results['parsed']
results['parsed']

ResultSchema(expenses=[Expense of 20000 on 2025-05-28, Expense of 5000 on 2025-05-28, Expense of 10000 on 2025-05-28, Expense of 30000 on 2025-05-27])

In [9]:
for expense in results.expenses:
    print(expense)

AttributeError: 'dict' object has no attribute 'expenses'